<a href="https://colab.research.google.com/github/oops-internet/AED/blob/main/APP_FInal_transcripci%C3%B3n_Audio_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Instalar Whisper
import subprocess
from tqdm import tqdm

# Función para instalar WhisperX con barra de progreso
def install_whisperx_with_progress():
    # Comando de instalación
    command = ["pip", "install", "git+https://github.com/m-bain/whisperx.git", "--upgrade"]

    # Ejecutar el comando y mostrar la barra de progreso
    with tqdm(total=100, desc="Instalando WhisperX", position=0, leave=True) as pbar:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for line in process.stdout:
            pbar.update(1)

# Instalar WhisperX con barra de progreso
install_whisperx_with_progress()


Instalando WhisperX: 130it [00:12, 10.65it/s]


In [ ]:
# @title Transcripción de Audio y Video
import whisperx
import gc
from google.colab import files
import os
from moviepy.editor import VideoFileClip
import ipywidgets as widgets

# Obtener códigos de idioma aceptados por WhisperX
ACCEPTED_LANGUAGES = {
    'Español (es)': 'es',
    'Inglés (en)': 'en',
    'Francés (fr)': 'fr',
    'Portugués (pt)': 'pt',  # Agregar portugués a la lista
}

# Función para cargar y procesar el archivo de audio
def process_audio_file(audio_file, target_language):
    # Crear una subcarpeta con el nombre del archivo sin extensión
    folder_name = os.path.splitext(os.path.basename(audio_file))[0]
    folder_path = os.path.join("/content", folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # Mover el archivo subido a la subcarpeta
    new_audio_file = os.path.join(folder_path, os.path.basename(audio_file))
    os.rename(audio_file, new_audio_file)

    # Convertir el video a audio si es un archivo de video
    if new_audio_file.endswith((".mp4", ".mov")):
        video = VideoFileClip(new_audio_file)
        audio_clip = video.audio
        new_audio_file = os.path.join(folder_path, folder_name + ".mp3")
        audio_clip.write_audiofile(new_audio_file)

    device = "cuda"
    batch_size = 16
    compute_type = "float16"

    # Transcripción
    model = whisperx.load_model("large-v2", device, compute_type=compute_type)
    audio = whisperx.load_audio(new_audio_file)
    result = model.transcribe(audio, batch_size=batch_size, language=target_language)
    print(result["segments"])

    # Alineación
    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)
    print(result["segments"])

    # Guardar la transcripción en un archivo de texto
    transcription_text = "\n".join([segment['text'] for segment in result["segments"]])
    transcription_file = os.path.join(folder_path, "transcription.txt")
    with open(transcription_file, "w") as f:
        f.write(transcription_text)

    print("Transcripción completada y guardada en:", transcription_file)

    # Descargar el archivo de transcripción
    files.download(transcription_file)

# Función para ejecutar la transcripción después de seleccionar el idioma de destino
def execute_transcription(b):
    selected_language = language_dropdown.value
    target_language = ACCEPTED_LANGUAGES.get(selected_language, None)
    if target_language:
        process_audio_file(uploaded_audio_file, target_language)
    else:
        print(f"'{selected_language}' no es un idioma válido admitido por WhisperX.")

# Subir el archivo de audio
uploaded = files.upload()
uploaded_audio_file = next(iter(uploaded))

# Crear lista desplegable para seleccionar el idioma de destino
language_dropdown = widgets.Dropdown(
    options=list(ACCEPTED_LANGUAGES.keys()),
    description="Idioma destino:",
    style={'description_width': 'initial'},
    disabled=False,
)

# Mostrar lista desplegable y botón de ejecutar transcripción
execute_button = widgets.Button(description="Ejecutar transcripción")
execute_button.on_click(execute_transcription)
widgets.VBox([language_dropdown, execute_button])
